In [1]:
import importlib
import threading
import time
import json
import datetime
import os
import codecs
    
from tcoreapi_mq import * 
import tcoreapi_mq

from pathlib import Path

import shioaji as sj
import shioaji_login
# Need to reload this for some reason that I can't remember.
importlib.reload(shioaji_login)

<module 'shioaji_login' from 'C:\\Users\\pride829\\git\\shioaji-coverer\\shioaji_login.py'>

In [2]:
def report_time():
    global NQ_price
    
    while True:
        print("Time report:", datetime.datetime.now().strftime("%H:%M:%S"), ", price:", NQ_price)
        time.sleep(auto_order_time_report)

In [3]:
def write_log(text):
    """
    Write into log file.
    
    :param text: (str)
    :return: None
    """

    now = datetime.datetime.now()
    path = 'auto_order_logs'
    try:
        Path(path).mkdir(parents=True, exist_ok=True)
    except FileExistsError:
        # directory already exists
        pass
    
    log_name = now.strftime('%Y%m%d') + '.log'
    path = os.path.join(path, log_name)

    # In order to let json dumps chinese correctly, codecs is needed.
    # When ever use json dumps, specify ensure_ascii=False
    fp = codecs.open(path, 'a+', 'utf16')
    fp.write(str(datetime.datetime.now()) + ": " + text + "\n")
    fp.close()
    

In [4]:
def list_positions():
    """
    List all positions.
    :global param: positions
    """
    
    print('The position currently tracking:')

    if not positions:
        print('Empty.')

    for p in positions:

        if(p[0] == 1):
            action_text = "Long"
        else:
            action_text = "Short"

        print(f'[Type: {action_text}, quantity: {p[1]}, deal price: {p[2]}, best price: {p[3]}, cover order had been sent: {p[4]}]')
    

In [5]:
def send_test_msg(
    price,
    quantity,
    action,
    stat=sj.constant.OrderState.FDeal,
    code='MXF',
    delivery_month='202201',
    security_type='FUT'
):
    """
    For test purpose.
    """
    # Testing with msg

    msg = {}
    msg['price'] = price
    msg['quantity'] = quantity
    msg['action'] = action
    msg['code'] = code
    msg['delivery_month'] = delivery_month
    msg["security_type"] = security_type

    place_cb(stat, msg)

In [6]:
def place_order(quantity, action):
    """
    Place a FOK order with price=MKT.
    :param: action = sj.constant.Action.Buy or sj.constant.Action.Sell
    :global param: api (shioaji.shioaji.Shioaji)
    :return: None
    """
    
    global auto_order_testing_day, auto_order_testing_night, positions
    
    auto_order_testing = True
    
    now = datetime.datetime.now()
    
    if now.time() < datetime.time(5, 0, 0) or now.time() >= datetime.time(15, 0, 0):
        auto_order_testing = auto_order_testing_night
    elif now.time() >= datetime.time(8, 45, 0) and now.time() < datetime.time(13, 45, 0):
        auto_order_testing = auto_order_testing_day
    
    if positions:
        print('***')
        log_msg = f'A order with action={action}, quantity={quantity} should be place, but there is still position in hold, \
so the order will not be sent.'
        
        print(log_msg)
        write_log(log_msg)
        
        list_positions()
        print('***')
        time.sleep(10)
        return
    
    if not auto_order_testing:
        fut_order = api.Order(
            action=action,
            price=0,
            quantity=quantity,
            price_type=sj.constant.FuturesPriceType.MKT,
            order_type=sj.constant.FuturesOrderType.FOK, 
            octype=sj.constant.FuturesOCType.Auto,
            account=api.futopt_account
        )

        # Placing order
        trade = api.place_order(contract, fut_order)

        print('***')
        log_msg = f'A FOK order with action={action}, quantity={quantity} has been placed!'
        print(log_msg)
        write_log(log_msg)
        print(f'Trade msg: {trade.status.msg}')
        write_log(f'Trade msg: {trade.status.msg}')
        print('***\n')
    else:
        print('***')
        log_msg = f'A TESTING FOK order with action={action}, quantity={quantity} has been placed!'
        print(log_msg)
        write_log(log_msg)
        print('***\n')
        
    time.sleep(10)

In [7]:
def get_future_code(future_name):
    """
    Return future code based on future name
    
    :param future_name: (str)
    
    :return: future_code (str)
    """
    
    now = datetime.datetime.now()
    month = now.month
    year = now.year
    first_weekday = now.replace(day=1).weekday()
    
    # Calculate the day of the third wednesday
    if(first_weekday < 3):
        third_wednesday = 17 - first_weekday
    else:
        third_wednesday = 24 - first_weekday
    
    if(
        now.day == third_wednesday and now.time() > datetime.time(13, 30) or
        now.day > third_wednesday
    ):
        month = month + 1
        if(month == 13):
            month = 1
            year += 1
    
    month_to_code = '0ABCDEFGHIJKL'
    
    future_code = future_name
    future_code += month_to_code[month]
    future_code += str(year%10)
    
    return future_code

In [8]:
def update_config():
    
    global future_name, future_code
    global order_quantity
    global auto_order_consec_tick, auto_order_time, auto_buy_trigger, auto_sell_trigger
    global auto_order_testing_day, auto_order_testing_night
    global auto_order_time_report
    
    pre_order_quantity = None
    pre_future_code = pre_auto_order_time = None
    pre_auto_buy_trigger = pre_auto_sell_trigger = None
    pre_auto_order_testing_day = pre_auto_order_testing_night = None
    pre_auto_order_consec_tick = pre_auto_order_time_report = None
    
    
    
    while(True):
        
        with open('auto_order_config.json') as f:
            config_data = json.load(f)
            
            future_name = config_data['future_name']
            order_quantity = int(config_data['order_quantity'])
            auto_order_time = float(config_data['auto_order_time'])
            auto_buy_trigger = float(config_data['auto_buy_trigger'])
            auto_sell_trigger = float(config_data['auto_sell_trigger'])
            if config_data['auto_order_testing_day'].lower() == "false":
                auto_order_testing_day = False
            else:
                auto_order_testing_day = True
                
            if config_data['auto_order_testing_night'].lower() == "false":
                auto_order_testing_night = False
            else:
                auto_order_testing_night = True
                
            auto_order_consec_tick = int(config_data['auto_order_consec_tick'])
            auto_order_time_report = float(config_data['auto_order_time_report'])
            
            # If auto_recent_future, get the most recent future code.
            if(config_data['auto_recent_future'].lower() == 'true'):
                future_code = get_future_code(future_name)
            else:
                future_code = config_data['future_code']
        
            if(pre_future_code != future_code):
                print(f'Future code has been set to {future_code}')
                pre_future_code = future_code
            
            if(pre_order_quantity != order_quantity):
                print(f'Order quantity has been set to {order_quantity}')
                pre_order_quantity = order_quantity
            
            if(pre_auto_order_time != auto_order_time):
                print(f'Auto order time has been set to {auto_order_time}')
                pre_auto_order_time = auto_order_time
            
            if(pre_auto_order_consec_tick != auto_order_consec_tick):
                print(f'Auto consecutive tick has been set to {auto_order_consec_tick}')
                pre_auto_order_consec_tick = auto_order_consec_tick
            
            if(pre_auto_buy_trigger != auto_buy_trigger):
                print(f'Auto buy trigger has been set to {auto_buy_trigger}')
                pre_auto_buy_trigger = auto_buy_trigger
                
            if(pre_auto_sell_trigger != auto_sell_trigger):
                print(f'Auto sell trigger has been set to {auto_sell_trigger}')
                pre_auto_sell_trigger = auto_sell_trigger
                
            if(pre_auto_order_testing_day != auto_order_testing_day):
                print(f'Auto order testing day has been set to {auto_order_testing_day}')
                pre_auto_order_testing_day = auto_order_testing_day
                
            if(pre_auto_order_testing_night != auto_order_testing_night):
                print(f'Auto order testing night has been set to {auto_order_testing_night}')
                pre_auto_order_testing_night = auto_order_testing_night
                
            if(pre_auto_order_time_report != auto_order_time_report):
                print(f'Auto order time report period has been set to {auto_order_time_report}')
                pre_auto_order_time_report = auto_order_time_report    
                
            time.sleep(1)

In [9]:
count = 0

In [10]:

# 在這裡下單
def OnRealTimeQuote(symbol):
    
    
    global count
    count += 1
    
    global NQ_price, price_history, trade_lock
    
    #print("OnRealTimeQuote: " + str(NQ_price))
    
    print(NQ_price, count)
    # price_history(不含最新的一個price)的長度即為中間間隔幾個tick的長度
    while len(price_history) > auto_order_consec_tick:
        del price_history[0]
    
    NQ_price = float(symbol['TradingPrice'])
    if NQ_price == 0:
        return
    
    
    price_history.append([NQ_price, datetime.datetime.now()])
    
    if len(price_history) < 2:
        return
    
    pre_price_diff = 0
    
    if trade_lock: # trade_lock=True時仍然會記錄價格，但不會判斷是否要trade
        return
    
    for i in range(len(price_history) - 1, 0, -1): # 從倒數第二個traverse到第一個
        
        price_diff = price_history[i][0] - price_history[i-1][0]
        
        
        #print(pre_price_diff, price_diff)
        if pre_price_diff > 0 and pre_price_diff <= price_diff:
            return
        elif pre_price_diff < 0 and pre_price_diff >= price_diff:
            return
        
        pre_price_diff = price_diff
        
        tick_diff = price_history[-1][0] - price_history[i-1][0]
        
        if (price_history[-1][1] - price_history[i-1][1]).total_seconds() > auto_order_time:
            # 超時
            break
        

        
        if price_diff > 0 and \
        tick_diff > auto_buy_trigger:
            msg_log = "A huge increasing in price has been detected!\n"
            msg_log += "NQ_Price " + str((price_history[-1][1] - price_history[i-1][1]).total_seconds()) + \
            " seconds ago: \n" + str(price_history[i-1][0]) + \
            ", " + str(price_history[i-1][1]) + \
            "\n"
            msg_log += "Ticks in between:\n"
            for j in range(i, len(price_history)-1):
                msg_log += str(price_history[j][0]) + ", " + str(price_history[j][1]) + "\n"
            msg_log += "NQ_Price now: \n" + str(price_history[-1][0]) + ", " + str(price_history[-1][1])
            write_log(msg_log)
            print(msg_log)
            trade_lock = True
            place_order(order_quantity, sj.constant.Action.Buy)
            price_history = []
            trade_lock = False
            return
        
        if price_diff < 0 and \
        tick_diff < -auto_sell_trigger:
            msg_log = "A huge decreasing in price has been detected!\n"
            msg_log += "NQ_Price " + str((price_history[-1][1] - price_history[i-1][1]).total_seconds()) + \
            " seconds ago: \n" + str(price_history[i-1][0]) + \
            ", " + str(price_history[i-1][1]) + \
            "\n"
            msg_log += "Ticks in between:\n"
            for j in range(i, len(price_history)-1):
                msg_log += str(price_history[j][0]) + ", " + str(price_history[j][1]) + "\n"
            msg_log += "NQ_Price now: \n" + str(price_history[-1][0]) + ", " + str(price_history[-1][1])
            write_log(msg_log)
            print(msg_log)
            trade_lock = True
            place_order(order_quantity, sj.constant.Action.Sell)
            price_history = []
            trade_lock = False
            return

In [11]:
def quote_sub_th(obj,sub_port,filter = ""):
    socket_sub = obj.context.socket(zmq.SUB)
    #socket_sub.RCVTIMEO=7000   #ZMQ超時設定
    socket_sub.connect("tcp://127.0.0.1:%s" % sub_port)
    socket_sub.setsockopt_string(zmq.SUBSCRIBE,filter)
    while(True):
        message = (socket_sub.recv()[:-1]).decode("utf-8")
        index =  re.search(":",message).span()[1]  # filter
        message = message[index:]
        message = json.loads(message)
        #for message in messages:
        if(message["DataType"]=="REALTIME"):
            OnRealTimeQuote(message["Quote"])
        elif(message["DataType"]=="GREEKS"):
            OnGreeks(message["Quote"])
        elif(message["DataType"]=="TICKS" or message["DataType"]=="1K" or message["DataType"]=="DK" ):
            #print("@@@@@@@@@@@@@@@@@@@@@@@",message)
            strQryIndex = ""
            while(True):
                s_history = obj.GetHistory(g_QuoteSession, message["Symbol"], message["DataType"], message["StartTime"], message["EndTime"], strQryIndex)
                historyData = s_history["HisData"]
                if len(historyData) == 0:
                    break
                last = ""
                for data in historyData:
                    last = data
                    print("歷史行情：Time:%s, Volume:%s, QryIndex:%s" % (data["Time"], data["Volume"], data["QryIndex"]))
                
                strQryIndex = last["QryIndex"]
                    
    return

In [12]:
def fill_positions(deal):
    """
    :global param positions: (list)
    
    :return: None
    """

    global positions
    # First check if the type and month match the tracking future.
    if(
        deal['code'] != contract['category'] or
        deal['delivery_month'] != contract['delivery_month'] or
        deal["security_type"] != 'FUT'
      ):
        print("This deal is not as same as the future currently tracking.")
        return
    
    price = int(deal['price'])
    quantity = int(deal['quantity'])
    try:
        if(deal['action'] == 'Buy'):
            action = 1
        elif(deal['action'] == 'Sell'):
            action = -1
        else:
            raise ValueError('The action of this deal is neither "Buy" or "Sell".')
    except ValueError as err:
        traceback.print_exc()
    
    if(action == 1):
        action_text = "Long"
    else:
        action_text = "Short"
    
    # While there are still some positions and it is the oppsite of the deal:
    ori_quantity = quantity
    while(positions and positions[0][0] == -action and quantity > 0):
        
        if(positions[0][1] > quantity):
            positions[0][1] -= quantity
            quantity = 0
            # The deal has been recorded, exit the function
            break
        else:
            quantity -= positions[0][1]
            del positions[0]

    print('***')
    log_msg = f'A position with type={action_text}, quantity={ori_quantity}, price={price} has been recorded!'
    print(log_msg)
    write_log(log_msg)
    print('***\n')
    
    if (quantity > 0):
        
        # Ensure the data type is int
        
        
        if(action == 1):
            positions.append([action, int(quantity), int(price), int(price), False])
            positions = sorted(positions, key=lambda p: p[2], reverse=False)
        else:
            positions.append([action, int(quantity), int(price), int(price), False])
            positions = sorted(positions, key=lambda p: p[2], reverse=True)

        print('***')
        log_msg = f'A position with type={action_text}, quantity={quantity}, price={price} has been added to the track list!'
        print(log_msg)
        write_log(log_msg)
        print('***\n')

In [13]:
msg_list = []

def place_cb(stat, msg):
    """
    Called every time an order or a deal has been detected.
    
    :global param: msg_list ()
    """
    
    global msg_list
    
    if(stat == sj.constant.OrderState.FOrder):
        print('An order has been detected.')
        print(f'op_msg: \"{msg["operation"]["op_msg"]}\"')
        write_log('stat: ' + stat + '\nmsg: ' + json.dumps(msg, ensure_ascii=False) )
        msg_list.append(msg)
    elif(stat == sj.constant.OrderState.FDeal):
        print('A deal has been detected.')
        print(f'Deal information: code:{msg["code"]}, action:{msg["action"]}, price:{msg["price"]}, quantity:{msg["quantity"]}')
        print(f'Delivery month:{msg["delivery_month"]}, security type: {msg["security_type"]}')
        write_log('stat: ' + stat + '\nmsg: ' + json.dumps(msg, ensure_ascii=False) )
        msg_list.append(msg)
        fill_positions(msg)

In [14]:
future_name = future_code = None

order_quantity = None

auto_order_time = auto_buy_trigger = auto_sell_trigger = None

auto_order_testing_day = auto_order_testing_night = auto_order_time_report = None

trade_lock = False

update_config_thread = threading.Thread(target = update_config)
update_config_thread.start()

time.sleep(3)

api = shioaji_login.login()

contract = api.Contracts.Futures[future_code]

api.set_order_callback(place_cb)

try:
    if(not contract):
        raise ValueError(f'Error: contract {future_code} does not exsits.')
except ValueError as err:
    traceback.print_exc()
    
positions = []

Future code has been set to MXFD2
Order quantity has been set to 1
Auto order time has been set to 10.0
Auto consecutive tick has been set to 2
Auto buy trigger has been set to 10.0
Auto sell trigger has been set to 10.0
Auto order testing day has been set to True
Auto order testing night has been set to True
Auto order time report period has been set to 1.0
Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
Login with ID N125685958
Login status: [FutureAccount(person_id='N125685958', broker_id='F002000', account_id='1645949', username='曾天裕'), Account(account_type=<AccountType.H: 'H'>, person_id='N125685958', broker_id='9A9X', account_id='0137289', username='曾天裕'), StockAccount(person_id='N125685958', broker_id='9A9X', account_id='0498332', signed=True, username='曾天裕\u3000\u3000')]
Activating CA at the path C:\Users\pride829

In [15]:
NQ_price = 0

price_history = []

g_QuoteZMQ = None
g_QuoteSession = ""

#登入(與 TOUCHANCE zmq 連線用，不可改)
g_QuoteZMQ = QuoteAPI("ZMQ","8076c9867a372d2a9a814ae710c256e2")
q_data = g_QuoteZMQ.Connect("51237")
print(q_data)

if q_data["Success"] != "OK":
    print("[quote]connection failed")

g_QuoteSession = q_data["SessionKey"]


#查詢指定合约訊息
quoteSymbol = "TC.F.CME.NQ.HOT"

print("Subscribing to CME.NQ.HOT")

t2 = threading.Thread(target = quote_sub_th,args=(g_QuoteZMQ,q_data["SubPort"],))
t2.start()
#實時行情訂閱
#解除訂閱
g_QuoteZMQ.UnsubQuote(g_QuoteSession, quoteSymbol)
#訂閱實時行情
g_QuoteZMQ.SubQuote(g_QuoteSession, quoteSymbol)

time.sleep(5)
report_time_thread = threading.Thread(target = report_time)
report_time_thread.start()


{'Reply': 'LOGIN', 'Success': 'OK', 'SessionKey': '313a67797c07fe4c7816eda6934c1429', 'SubPort': '50761'}
Subscribing to CME.NQ.HOT
0 1
13853.75 2
13853.75 3
13854.0 4
13852.25 5
13851.5 6
13849.25 7
13849.25 8
13848.0 9
13847.0 10
Time report: 21:39:47 , price: 13846.0


In [19]:
1

1

14133.25 9401
Time report: 23:18:43 , price: 14133.25
14133.25 9402
Time report: 23:18:44 , price: 14132.75
14132.75 9403
Time report: 23:18:45 , price: 14134.5
14134.5 9404
14134.5 9405
Time report: 23:18:46 , price: 14134.5
14134.5 9406
14135.25 9407
Time report: 23:18:47 , price: 14134.75
14134.75 9408
14134.5 9409
Time report: 23:18:48 , price: 14137.5
14137.5 9410
Time report: 23:18:49 , price: 14137.0
14137.0 9411
Time report: 23:18:50 , price: 14138.5
14138.5 9412
14137.75 9413
14137.75 9414
Time report: 23:18:51 , price: 14137.25
14137.25 9415
Time report: 23:18:52 , price: 14137.75
14137.75 9416
Time report: 23:18:53 , price: 14135.75
14135.75 9417
14136.0 9418
14136.75 9419
Time report: 23:18:54 , price: 14137.0
14137.0 9420
Time report: 23:18:55 , price: 14137.0
14137.0 9421
14137.0 9422
Time report: 23:18:56 , price: 14137.0
14137.0 9423
14136.75 9424
Time report: 23:18:57 , price: 14135.5
14135.5 9425
14135.25 9426
Time report: 23:18:58 , price: 14134.75
14134.75 9427
Time

14136.0 9620
14136.75 9621
Time report: 23:20:57 , price: 14136.25
14136.25 9622
Time report: 23:20:58 , price: 14136.5
14136.5 9623
14135.75 9624
14137.0 9625
Time report: 23:20:59 , price: 14136.75
14136.75 9626
14136.5 9627
14136.0 9628
Time report: 23:21:00 , price: 14136.0
14136.0 9629
14136.5 9630
Time report: 23:21:01 , price: 14137.0
Time report: 23:21:02 , price: 14137.0
14137.0 9631
14134.25 9632
Time report: 23:21:03 , price: 14134.75
Time report: 23:21:04 , price: 14134.75
14134.75 9633
14134.25 9634
Time report: 23:21:05 , price: 14134.25
14134.25 9635
14133.75 9636
14134.25 9637
Time report: 23:21:06 , price: 14134.0
14134.0 9638
Time report: 23:21:07 , price: 14134.25
14134.25 9639
Time report: 23:21:08 , price: 14134.5
14134.5 9640
14134.75 9641
Time report: 23:21:09 , price: 14133.75
14133.75 9642
14134.75 9643
Time report: 23:21:10 , price: 14134.0
14134.0 9644
14133.75 9645
14133.75 9646
Time report: 23:21:11 , price: 14133.5
14133.5 9647
Time report: 23:21:12 , pric

14153.25 9847
14155.0 9848
Time report: 23:23:08 , price: 14155.25
14155.25 9849
14153.75 9850
Time report: 23:23:09 , price: 14154.0
14154.0 9851
Time report: 23:23:10 , price: 14153.25
14153.25 9852
14154.0 9853
14154.25 9854
Time report: 23:23:11 , price: 14154.25
14154.25 9855
Time report: 23:23:12 , price: 14154.25
14154.25 9856
Time report: 23:23:13 , price: 14154.75
14154.75 9857
14155.0 9858
Time report: 23:23:14 , price: 14155.25
14155.25 9859
14156.0 9860
Time report: 23:23:15 , price: 14157.0
14157.0 9861
Time report: 23:23:16 , price: 14157.0
14157.0 9862
Time report: 23:23:17 , price: 14157.25
14157.25 9863
14156.5 9864
Time report: 23:23:18 , price: 14156.0
14156.0 9865
14156.0 9866
14156.25 9867
14156.25 9868
Time report: 23:23:19 , price: 14156.5
14156.5 9869
Time report: 23:23:20 , price: 14157.25
14157.25 9870
Time report: 23:23:21 , price: 14156.75
14156.75 9871
14158.0 9872
14158.25 9873
Time report: 23:23:22 , price: 14157.25
14157.25 9874
Time report: 23:23:23 , p

14166.0 10074
Time report: 23:25:17 , price: 14164.75
14164.75 10075
14167.5 10076
Time report: 23:25:18 , price: 14167.25
14167.25 10077
Time report: 23:25:19 , price: 14166.75
14166.75 10078
14166.75 10079
Time report: 23:25:20 , price: 14166.5
14166.5 10080
14167.75 10081
Time report: 23:25:21 , price: 14167.75
14167.75 10082
14167.75 10083
Time report: 23:25:22 , price: 14168.25
14168.25 10084
14167.25 10085
Time report: 23:25:23 , price: 14166.75
14166.75 10086
Time report: 23:25:24 , price: 14166.75
Time report: 23:25:25 , price: 14166.75
14166.75 10087
Time report: 23:25:26 , price: 14164.5
14164.5 10088
14164.25 10089
Time report: 23:25:27 , price: 14164.75
14164.75 10090
14164.5 10091
14164.5 10092
Time report: 23:25:28 , price: 14163.75
14163.75 10093
14164.25 10094
Time report: 23:25:29 , price: 14164.25
14164.25 10095
14165.5 10096
Time report: 23:25:30 , price: 14166.5
14166.5 10097
14166.25 10098
Time report: 23:25:31 , price: 14166.25
14166.25 10099
Time report: 23:25:32

...

14165.0 11159
Time report: 23:36:07 , price: 14167.25
14167.25 11160
14168.0 11161
Time report: 23:36:08 , price: 14169.0
14169.0 11162
14169.0 11163
Time report: 23:36:09 , price: 14169.25
14169.25 11164
14167.75 11165
Time report: 23:36:10 , price: 14167.25
14167.25 11166
14167.0 11167
Time report: 23:36:11 , price: 14166.25
14166.25 11168
14165.75 11169
14165.25 11170
Time report: 23:36:12 , price: 14164.75
14164.75 11171
Time report: 23:36:13 , price: 14164.5
14164.5 11172
14165.0 11173
14163.25 11174
Time report: 23:36:14 , price: 14163.25
14163.25 11175
14163.25 11176
Time report: 23:36:15 , price: 14163.25
14163.25 11177
Time report: 23:36:16 , price: 14162.5
14162.5 11178
14161.75 11179
14161.0 11180
Time report: 23:36:17 , price: 14161.0
14161.0 11181
Time report: 23:36:18 , price: 14160.5
14160.5 11182
14159.75 11183
14159.25 11184
Time report: 23:36:19 , price: 14159.25
14159.25 11185
14158.25 11186
Time report: 23:36:20 , price: 14158.25
14158.25 11187
14158.25 11188
Time r

14166.5 11394
Time report: 23:38:09 , price: 14166.25
14166.25 11395
14166.5 11396
Time report: 23:38:10 , price: 14166.5
14166.5 11397
14165.75 11398
Time report: 23:38:11 , price: 14166.75
14166.75 11399
14166.75 11400
Time report: 23:38:12 , price: 14166.25
14166.25 11401
14166.25 11402
Time report: 23:38:13 , price: 14165.75
14165.75 11403
14165.75 11404
Time report: 23:38:14 , price: 14164.75
14164.75 11405
Time report: 23:38:15 , price: 14164.75
14164.75 11406
14164.5 11407
Time report: 23:38:16 , price: 14165.25
14165.25 11408
14165.25 11409
Time report: 23:38:17 , price: 14165.25
14165.25 11410
14164.5 11411
Time report: 23:38:18 , price: 14164.75
14164.75 11412
14165.25 11413
Time report: 23:38:19 , price: 14163.5
14163.5 11414
14162.75 11415
Time report: 23:38:20 , price: 14162.75
14162.75 11416
14164.0 11417
14163.5 11418
Time report: 23:38:21 , price: 14163.5
14163.5 11419
14162.75 11420
Time report: 23:38:22 , price: 14163.5
14163.5 11421
Time report: 23:38:23 , price: 141

Time report: 23:40:12 , price: 14178.75
14178.75 11626
14178.75 11627
Time report: 23:40:13 , price: 14178.75
14178.75 11628
14179.25 11629
Time report: 23:40:14 , price: 14178.0
14178.0 11630
Time report: 23:40:15 , price: 14177.0
14177.0 11631
14177.5 11632
14177.25 11633
14177.75 11634
Time report: 23:40:16 , price: 14178.0
14178.0 11635
14177.5 11636
14177.0 11637
Time report: 23:40:17 , price: 14177.0
14177.0 11638
Time report: 23:40:18 , price: 14177.25
14177.25 11639
Time report: 23:40:19 , price: 14176.5
14176.5 11640
14175.0 11641
Time report: 23:40:20 , price: 14174.5
14174.5 11642
14174.25 11643
14175.25 11644
Time report: 23:40:21 , price: 14175.75
14175.75 11645
14176.25 11646
Time report: 23:40:22 , price: 14177.25
14177.25 11647
Time report: 23:40:23 , price: 14177.0
14177.0 11648
14177.0 11649
14177.75 11650
Time report: 23:40:24 , price: 14177.75
14177.75 11651
14177.25 11652
Time report: 23:40:25 , price: 14176.75
14176.75 11653
14177.25 11654
Time report: 23:40:26 , 

14178.25 11861
14178.25 11862
14178.0 11863
Time report: 23:42:15 , price: 14177.5
14177.5 11864
14177.75 11865
14178.0 11866
Time report: 23:42:16 , price: 14178.0
14178.0 11867
14178.0 11868
Time report: 23:42:17 , price: 14177.75
14177.75 11869
14178.0 11870
Time report: 23:42:18 , price: 14177.75
14177.75 11871
Time report: 23:42:19 , price: 14179.5
14179.5 11872
14181.0 11873
Time report: 23:42:20 , price: 14180.0
14180.0 11874
14181.0 11875
Time report: 23:42:21 , price: 14180.25
14180.25 11876
14179.5 11877
Time report: 23:42:22 , price: 14179.5
14179.5 11878
14179.25 11879
Time report: 23:42:23 , price: 14179.25
14179.25 11880
14179.25 11881
Time report: 23:42:24 , price: 14179.25
14179.25 11882
14178.75 11883
14179.0 11884
Time report: 23:42:25 , price: 14179.0
14179.0 11885
14178.5 11886
Time report: 23:42:26 , price: 14179.5
14179.5 11887
Time report: 23:42:27 , price: 14180.0
14180.0 11888
14180.0 11889
Time report: 23:42:28 , price: 14180.75
14180.75 11890
14180.75 11891
T

14182.5 12092
14182.5 12093
Time report: 23:44:20 , price: 14183.25
14183.25 12094
Time report: 23:44:21 , price: 14183.0
14183.0 12095
14183.5 12096
Time report: 23:44:22 , price: 14182.5
14182.5 12097
14182.0 12098
Time report: 23:44:23 , price: 14181.5
14181.5 12099
14180.75 12100
Time report: 23:44:24 , price: 14180.75
14180.75 12101
14180.75 12102
Time report: 23:44:25 , price: 14180.75
14180.75 12103
Time report: 23:44:26 , price: 14180.0
14180.0 12104
14180.0 12105
Time report: 23:44:27 , price: 14181.0
14181.0 12106
14180.5 12107
14180.5 12108
14180.25 12109
Time report: 23:44:28 , price: 14180.25
14180.25 12110
Time report: 23:44:29 , price: 14180.0
14180.0 12111
14179.5 12112
14179.75 12113
Time report: 23:44:30 , price: 14179.75
14179.75 12114
14179.25 12115
Time report: 23:44:31 , price: 14179.75
14179.75 12116
Time report: 23:44:32 , price: 14180.25
14180.25 12117
14180.5 12118
Time report: 23:44:33 , price: 14180.5
14180.5 12119
14180.75 12120
14180.75 12121
Time report: 

In [16]:
1/0 # Testing

13846.0 11
Time report: 21:39:48 , price: 13846.0


ZeroDivisionError: division by zero

In [ ]:
price_history = []

symbol0 = {'TradingPrice': 17000}
OnRealTimeQuote(symbol0)
time.sleep(0.25)
symbol1 = {'TradingPrice': 17003}
OnRealTimeQuote(symbol1)
time.sleep(0.25)
symbol2 = {'TradingPrice': 17007}
OnRealTimeQuote(symbol2)
time.sleep(0.25)
symbol3 = {'TradingPrice': 17012}
OnRealTimeQuote(symbol3)
time.sleep(0.25)
symbol4 = {'TradingPrice': 17020}
OnRealTimeQuote(symbol4)
"""
symbol5 = {'TradingPrice': 17020}
OnRealTimeQuote(symbol5)
time.sleep(0.25)
symbol6 = {'TradingPrice': 17019}
OnRealTimeQuote(symbol6)
time.sleep(0.25)
symbol7 = {'TradingPrice': 17017}
OnRealTimeQuote(symbol7)
time.sleep(0.25)
symbol8 = {'TradingPrice': 17012}
OnRealTimeQuote(symbol8)
time.sleep(0.25)
symbol9 = {'TradingPrice': 17004}
OnRealTimeQuote(symbol9)
"""

In [ ]:
price_history

In [ ]:
market_price = 17000

send_test_msg(
    int(market_price),
    1,
    sj.constant.Action.Buy,
    stat=sj.constant.OrderState.FDeal,
    code='MXF',
    delivery_month='202204',
    security_type='FUT'
)

In [ ]:
place_order(1, sj.constant.Action.Buy)

In [ ]:
list_positions()

In [ ]:
auto_order_testing

In [ ]:
# Touchance傳資料
# 過一分鐘自動回報

"""
台灣是UTC＋8
那斯達克: 標準時間（EST）為UTC-5，夏令時間（EDT）為UTC-4。開盤時間：週一至週五上午9:30至下午4：00
也就是說開盤時間為台灣時間周一22:30~週五17:00，平常為22:30~17:00


"""

13846.0 12
Time report: 21:39:49 , price: 13846.75
13846.75 13
Time report: 21:39:50 , price: 13849.0
13849.0 14
13851.75 15
Time report: 21:39:51 , price: 13853.5
13853.5 16
13854.25 17
Time report: 21:39:52 , price: 13854.5
13854.5 18
Time report: 21:39:53 , price: 13854.0
13854.0 19
13853.0 20
Time report: 21:39:54 , price: 13850.0
Time report: 21:39:55 , price: 13850.0
13850.0 21
Time report: 21:39:56 , price: 13849.25
13849.25 22
Time report: 21:39:57 , price: 13850.0
13850.0 23
Time report: 21:39:58 , price: 13848.25
13848.25 24
Time report: 21:39:59 , price: 13847.75
13847.75 25
Time report: 21:40:00 , price: 13849.0
Time report: 21:40:01 , price: 13849.0
13849.0 26
13848.75 27
Time report: 21:40:02 , price: 13849.0
13849.0 28
Time report: 21:40:03 , price: 13847.25
13847.25 29
Time report: 21:40:04 , price: 13843.25
Time report: 21:40:05 , price: 13843.25
13843.25 30
Time report: 21:40:06 , price: 13843.25
13843.25 31
Time report: 21:40:07 , price: 13838.0
13838.0 32
Time repor